In [20]:
import numpy as np
import xgboost as xgb
import os
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
directory = "data/"
labels = {}
data = []
i = 0
for root, dirs, files in os.walk(directory):
    for file in files:
        path = os.path.join(root, file)
        if(file.endswith(".txt")):
            labels[i] = file.replace(".txt","")            
#             data.append(pd.read_csv(path,header=None).drop(0,axis=1).drop_duplicates())
            i += 1
            
# for i in range(len(labels)):
#     data[i][290] = i   
# pd.concat(data).to_csv("data.txt",header=None)

In [3]:
df = pd.read_csv("data.txt",header=None).drop(0,axis=1)

In [11]:
X = df.drop(idxsToRemove+[290],axis=1).values
y = df[290].values

In [18]:
X.shape

(8763, 249)

In [7]:
indxs = np.array([10,13])
num_ids = indxs.shape[0]
dist = 14*2+1

idxsToRemove = []

for idx in indxs:
    
    for i in range(10):

        idxsToRemove.append(2*idx+i*dist+1)
        idxsToRemove.append(2*idx+i*dist+2)


In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

lamdas = np.logspace
score_best = -10e15
for lamda in lamdas:
        print(lamda)
        clf = xgb.XGBRegressor(n_estimators=1000, learning_rate=1, gamma=0, n_jobs=4, reg_lambda=lamda,
                           colsample_bytree=1, max_depth=7)   
        clf.fit(X_train,y_train)
        test_score = accuracy_score(y_test,clf.predict(X_test))
        if(test_score>score_best):
            print(test_score)
            score_best = test_score
            best_lamda = lamda


In [14]:
clf = xgb.XGBClassifier(n_estimators=1000, learning_rate=0.1, n_jobs=4,max_depth=7) 
clf.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=4, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [15]:
y_pred = clf.predict(X)

/home/k_p0w3r/Programs/Anaconda/envs/tfenv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:
accuracy_score(y,y_pred)

1.0

In [17]:
pickle.dump(clf, open("clf.dat", "wb"),protocol=2)

In [22]:
loaded_model = pickle.load(open("clf.dat"", "rb"))

In [23]:
y_pred = loaded_model.predict(X)

/home/k_p0w3r/Programs/Anaconda/envs/tfenv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [24]:
accuracy_score(y,y_pred)

1.0

In [26]:
labels

{0: 'jogging', 1: 'waving', 2: 'running', 3: 'walking', 4: 'clapping'}

In [27]:
pickle.dump(labels, open("labels.dat", "wb"),protocol=2)

In [28]:
asd = pickle.load(open("labels.dat", "rb"))

In [29]:
asd

{0: 'jogging', 1: 'waving', 2: 'running', 3: 'walking', 4: 'clapping'}